In [ ]:
!pip install radonpy-pypi deepchem mordred

In [ ]:
from rdkit.Chem import rdchem
from radonpy.core.utils import mol_from_smiles
from radonpy.core.poly import polymerize_rw, terminate_mols
from typing import Literal


def build_polymer(
    repeat_units: str,
    head_smiles: str,
    tail_smiles: str,
    n_repreat_unit: int,
    tacticity: Literal["atactic", "isotactic", "syndiotactic"] = "atactic",
) -> rdchem.Mol:
    head_mol = mol_from_smiles(head_smiles)
    tail_mol = mol_from_smiles(tail_smiles)
    repeat_units_mol = mol_from_smiles(repeat_units)
    polymer = polymerize_rw(repeat_units_mol, n_repreat_unit, tacticity=tacticity)
    polymer = terminate_mols(polymer, head_mol, tail_mol)
    return polymer

In [ ]:
import deepchem as dc

repeat_units = "*CC*"
begin_smiles = "C*"
end_smiles = "C*"

polymer = build_polymer(repeat_units, begin_smiles, end_smiles, 3)
mordred_featurizer = dc.feat.MordredDescriptors(ignore_3D=True)
morderd_features = mordred_featurizer.featurize(polymer)
maccskey_featurizer = dc.feat.MACCSKeysFingerprint()
maccskey_features = maccskey_featurizer.featurize(polymer)
circular_featurizer = dc.feat.CircularFingerprint(size=2048, radius=4)
circular_features = circular_featurizer.featurize(polymer)

In [ ]:
from openff.interchange import Interchange
from openff.toolkit import ForceField, Molecule

off_mol = Molecule.from_rdkit(polymer)
off_mol.generate_conformers()

ff = ForceField("openff-2.0.0.offxml")
interchange = Interchange.from_smirnoff(topology=[off_mol], force_field=ff)
interchange.to_lammps_datafile("polymer.data")

lmp_input = """
units real
atom_style full

dimension 3
boundary p p p

bond_style hybrid harmonic
angle_style hybrid harmonic
dihedral_style hybrid fourier
improper_style cvff
special_bonds lj 0.0 0.0 0.5 coul 0.0 0.0 0.8333333333 
pair_style lj/cut/coul/cut 12.0 12.0
pair_modify mix arithmetic tail yes

read_data polymer.data

compute DIPOLE all dipole
thermo_style custom step time ebond eangle edihed eimp epair evdwl ecoul elong etail pe c_DIPOLE[0] c_DIPOLE[1] c_DIPOLE[2]
timestep 1.0
fix NVT all nvt temp 300.0 300.0 100.0
thermo 1000
run 1000000

"""

with open("dipole.in", "w") as fi:
    fi.write(lmp_input)

!lmp_mpi -i dipole.in -log dipole.log

In [ ]:
import psi4
from rdkit.Chem import AllChem
import numpy as np

repeat_units = "*CC*"
begin_smiles = "C*"
end_smiles = "C*"

polymer = build_polymer(repeat_units, begin_smiles, end_smiles, 3)
AllChem.UFFOptimizeMolecule(polymer)
elements = [atom.GetSymbol() for atom in polymer.GetAtoms()]
positions = polymer.GetConformer(0).GetPositions()
psi4_geom_input = []
for elem, pos in zip(elements, positions):
    psi4_geom_input.append(f"{elem} {pos[0]} {pos[1]} {pos[2]}")
psi4_geom_input_join = "\n".join(psi4_geom_input)
molecule = psi4.geometry(psi4_geom_input_join)
opt_energy, opt_wfn, history = psi4.optimize(
    "b3lyp/6-31g", return_wfn=True, return_history=True
)
psi4.oeprop(opt_wfn, "DIPOLE")
dipole = psi4.variable("SCF DIPOLE")

psi4.oeprop(opt_wfn, "DIPOLE")
dipole = psi4.variable("SCF DIPOLE")
dipole_magnitude = np.sqrt(np.sum(dipole**2))